In [1]:
from huggingface_hub import login
login(new_session=False)

import torch
import pandas as pd
from sentence_transformers import CrossEncoder
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re
import json
from google.colab import drive

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
NLLB_MODEL_ID = "facebook/nllb-200-distilled-600M"  # smaller/faster
NLLB_BATCH_SIZE = 64
NLLB_MAX_LENGTH = 128

NLI_MODEL_ID = "cross-encoder/nli-deberta-v3-base"
NLI_BATCH_SIZE = 64
NLI_MAX_LENGTH = 384

In [3]:
# -------------------------------
# LOAD NLI MODEL
# -------------------------------
print(f"🚀 Loading NLI model: {NLI_MODEL_ID}...")
nli_model = CrossEncoder(NLI_MODEL_ID, device="cuda", max_length=NLI_MAX_LENGTH)
nli_model.model.half()
nli_model.model.eval()

# -------------------------------
# LOAD NLLB MODEL
# -------------------------------
print(f"🌍 Loading NLLB model: {NLLB_MODEL_ID}...")
nllb_tokenizer = AutoTokenizer.from_pretrained(NLLB_MODEL_ID)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(
    NLLB_MODEL_ID,
    torch_dtype=torch.float16
).cuda()
nllb_model.eval()


🚀 Loading NLI model: cross-encoder/nli-deberta-v3-base...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

In [16]:
def normalize_answer(text):
    text = re.sub(r"[\*\-•]", "", text)
    text = text.replace(":", " was")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def translate_batch_nllb(texts, src_lang_code, tgt_lang_code="eng_Latn", batch_size=NLLB_BATCH_SIZE, max_length=NLLB_MAX_LENGTH):
    outputs = []

    # Prepend the target language to each text
    texts_with_lang = [f"{tgt_lang_code}: {t}" if isinstance(t, str) else "" for t in texts]

    for i in tqdm(range(0, len(texts_with_lang), batch_size), desc="Translating", total=(len(texts_with_lang)+batch_size-1)//batch_size):
        batch = texts_with_lang[i:i+batch_size]

        inputs = nllb_tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to("cuda")

        with torch.no_grad():
            generated = nllb_model.generate(
                **inputs,
                max_length=max_length,
                num_beams=2,
                do_sample=False
            )

        decoded = nllb_tokenizer.batch_decode(generated, skip_special_tokens=True)
        outputs.extend(decoded)

    return outputs




def evaluate_correctness(df, pred_col, gold_col):
    if pred_col not in df.columns or gold_col not in df.columns:
        raise ValueError(f"Missing columns. Found: {df.columns.tolist()}")

    # Correct NLI: Premise = GOLD, Hypothesis = MODEL OUTPUT
    pairs = df[[gold_col, pred_col]].values.tolist()
    norm_pairs = [(gold, normalize_answer(pred)) for gold, pred in pairs]

    print(f"⚡ Running NLI on {len(pairs)} examples...")
    scores = nli_model.predict(pairs, batch_size=NLI_BATCH_SIZE, show_progress_bar=True)
    norm_scores = nli_model.predict(norm_pairs, batch_size=NLI_BATCH_SIZE, show_progress_bar=True)

    labels = scores.argmax(axis=1)
    norm_labels = norm_scores.argmax(axis=1)
    id2label = nli_model.model.config.id2label

    df["nli_label"] = [id2label[l] for l in labels]
    df["nli_label_norm"] = [id2label[l] for l in norm_labels]

    df["hallucinated"] = df["nli_label"] == "contradiction"
    df["hallucinated_norm"] = df["nli_label_norm"] == "contradiction"

    total = len(df)
    halluc_rate = df["hallucinated"].mean() * 100
    norm_halluc_rate = df["hallucinated_norm"].mean() * 100

    print("\n✅ DONE")
    print(f"📊 Total rows: {total}")
    print(f"🚨 Hallucination rate: {halluc_rate:.2f}%")
    print(f"🚨 Hallucination rate (post norm): {norm_halluc_rate:.2f}%")
    return df


In [5]:
drive.mount('/content/drive')

langs = {
    "hi": "hin_Latn",
    "ka": "kan_Latn",
    "ma": "mar_Latn",
    "ta": "tam_Latn"
}

models = ["aya", "gemma", "sarvam"]

main_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/{m}/{m}_results_{l}.csv"
translated_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Translated/{m}/{m}_results_{l}_translated_en.csv"

# -------------------------------
# LOAD ENGLISH GOLD ANSWERS
# -------------------------------
json_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/cleaned_data/english_cleaned.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []
for article in data["data"]:
    for para in article.get("paragraphs", []):
        for qa in para.get("qas", []):
            qid = qa.get("id")
            answer = qa["answers"][0]["text"] if qa.get("answers") else ""
            rows.append({"id": qid, "Answer": answer})

english_df = pd.DataFrame(rows)

Mounted at /content/drive


In [8]:
def translate_and_nli(m, lang_code, english_df):
    csv_path = main_path.format(m=m, l=lang_code)
    print(f"🚀 Loading {csv_path}")
    df = pd.read_csv(csv_path)

    # --- SAFELY ADD GOLD ANSWERS ---
    df = df.merge(english_df, on="id", how="left")
    df.rename(columns={"Answer": "en_gold"}, inplace=True)

    # --- TRANSLATION ---
    print(f"🌍 Translating {m} {lang_code} with NLLB...")
    df["translated_answers"] = translate_batch_nllb(
        df["model_answer"].tolist(),
        src_lang_code=langs[lang_code]
    )

    # Save translated CSV
    out_trans_path = translated_path.format(m=m, l=lang_code)
    df.to_csv(out_trans_path, index=False)
    print(f"💾 Translated CSV saved: {out_trans_path}")

    # --- NLI ---
    print(f"⚡ Running NLI on {m} {lang_code}...")
    df = evaluate_correctness(df, "translated_answers", "en_gold")

    out_nli_path = out_trans_path.replace(".csv", "_nli_graded.csv")
    df.to_csv(out_nli_path, index=False)
    print(f"💾 NLI graded CSV saved: {out_nli_path}")


In [17]:
for m in models:
    for lang_code in langs.keys():
        translate_and_nli(m, lang_code, english_df)


🚀 Loading /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/aya/aya_results_hi.csv
🌍 Translating aya hi with NLLB...


Translating:   0%|          | 0/4 [00:00<?, ?it/s]

💾 Translated CSV saved: /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Translated/aya/aya_results_hi_translated_en.csv
⚡ Running NLI on aya hi...
⚡ Running NLI on 200 examples...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Translating:   0%|          | 0/4 [00:00<?, ?it/s]

💾 Translated CSV saved: /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Translated/sarvam/sarvam_results_ta_translated_en.csv
⚡ Running NLI on sarvam ta...
⚡ Running NLI on 200 examples...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]


✅ DONE
📊 Total rows: 200
🚨 Hallucination rate: 44.50%
🚨 Hallucination rate (post norm): 44.50%
💾 NLI graded CSV saved: /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Translated/sarvam/sarvam_results_ta_translated_en_nli_graded.csv


english

In [18]:
def nli_for_english_csv(m, lang_code="en", english_df=english_df):
    # Load the model CSV (already in English)
    csv_path = main_path.format(m=m, l=lang_code)
    print(f"🚀 Loading {csv_path}")
    df = pd.read_csv(csv_path)

    # Merge with English gold answers by ID to ensure alignment
    df = df.merge(english_df, on="id", how="left")
    df.rename(columns={"Answer": "en_gold"}, inplace=True)

    # Run NLI directly
    print(f"⚡ Running NLI on {m} {lang_code}...")
    df = evaluate_correctness(df, "model_answer", "en_gold")

    # Save NLI-graded CSV
    out_nli_path = csv_path.replace(".csv", "_nli_graded.csv")
    df.to_csv(out_nli_path, index=False)
    print(f"💾 NLI graded CSV saved: {out_nli_path}")


In [19]:
for m in models:
    nli_for_english_csv(m, lang_code="en", english_df=english_df)


🚀 Loading /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/aya/aya_results_en.csv
⚡ Running NLI on aya en...
⚡ Running NLI on 200 examples...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]


✅ DONE
📊 Total rows: 200
🚨 Hallucination rate: 2.50%
🚨 Hallucination rate (post norm): 2.00%
💾 NLI graded CSV saved: /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/aya/aya_results_en_nli_graded.csv
🚀 Loading /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/gemma/gemma_results_en.csv
⚡ Running NLI on gemma en...
⚡ Running NLI on 200 examples...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]


✅ DONE
📊 Total rows: 200
🚨 Hallucination rate: 6.50%
🚨 Hallucination rate (post norm): 5.50%
💾 NLI graded CSV saved: /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/gemma/gemma_results_en_nli_graded.csv
🚀 Loading /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/sarvam/sarvam_results_en.csv
⚡ Running NLI on sarvam en...
⚡ Running NLI on 200 examples...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]


✅ DONE
📊 Total rows: 200
🚨 Hallucination rate: 9.00%
🚨 Hallucination rate (post norm): 9.00%
💾 NLI graded CSV saved: /content/drive/MyDrive/Project/Dataset/IndicSquad/Results/Vanilla-t200/sarvam/sarvam_results_en_nli_graded.csv
